In [1]:
import numpy as np
import random
import gym
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib
import pickle
import pandas as pd
from scipy.io import savemat, loadmat
import networkx as nx

In [2]:
def generate_traffic_matrix(node_reqs):
    probs = np.zeros([len(node_reqs),len(node_reqs)])
    for i in range(len(node_reqs)):
        for j in range(len(node_reqs)):
            if i < j:
                new_node_reqs = np.copy(node_reqs)
                new_node_reqs[i] = 0.
                new_node_reqs = new_node_reqs/np.sum(new_node_reqs)
                probs[i,j] = node_reqs[i] * new_node_reqs[j]
#             if i > j:
#                 new_node_reqs = np.copy(node_reqs)
#                 new_node_reqs[j] = 0.
#                 new_node_reqs = new_node_reqs/np.sum(new_node_reqs)
#                 probs[i,j] = node_reqs[i] * new_node_reqs[j]
    return probs
def traffic_mat_to_node_probs(traffic_matrix, num_nodes):
    node_probs= []
    col_probs =[]
    row_probs = []
    for row in range(num_nodes):
        row_probs.append(np.sum(traffic_matrix[row]))
    for col in range(num_nodes):
        col_probs.append(np.sum(traffic_matrix[:,col]))
    for node in range(num_nodes):
        node_row_prob = row_probs[node]
        node_col_prob = col_probs[node]
        node_probs.append((node_col_prob+node_col_prob)/2)
    return node_probs

Generation of uniform and perturbed uniform traffic matrices:

In [ ]:
uni_prob = 1/14
# node_request_probabilities = np.ones(14)*uni_prob + np.random.uniform(0,uni_prob*0.05, 14)

In [ ]:
node_request_probabilities_12_uni = np.ones(12)/12
pickle.dump(node_request_probabilities_12_uni, open("/Users/joshnevin/RL_FOCSLab/node_req_probs/12_node_uniform.pkl",'wb'))

In [3]:
node_request_probabilities_28_uni = np.ones(28)/28
pickle.dump(node_request_probabilities_28_uni, open("/Users/joshnevin/RL_FOCSLab/node_req_probs/28_node_uniform.pkl",'wb'))

In [4]:
sum(node_request_probabilities_28_uni)

0.9999999999999997

Generated perturbation on first run with seed of 9

In [ ]:
rng_np = np.random.default_rng(9)
pert = 0.5
node_request_probabilities = np.ones(14) + rng_np.uniform(-pert, pert, 14)

In [ ]:
node_request_probabilities = node_request_probabilities/np.sum(node_request_probabilities)

In [ ]:
np.sum(node_request_probabilities)

In [ ]:
node_request_probabilities

In [ ]:
pickle.dump(node_request_probabilities, open("/Users/joshnevin/RL_FOCSLab/node_req_probs/14_node_uniform_pert"+str(int(pert*100))+".pkl",'wb'))

In [ ]:
non_uni = pickle.load(open("/Users/joshnevin/RL_FOCSLab/node_req_probs/14_node_uniform_pert50.pkl",'rb'))
uni = pickle.load(open("/Users/joshnevin/RL_FOCSLab/node_req_probs/14_node_uniform.pkl",'rb'))
grav = pickle.load(open("/Users/joshnevin/RL_FOCSLab/node_req_probs/nsfnet_chen_gravity.pkl",'rb'))

In [ ]:
non_uniform_matrix = generate_traffic_matrix(non_uni)

In [ ]:
# uniform_matrix = (generate_traffic_matrix(uni).round(decimals=4)*10000).astype(dtype=int)
# non_uniform_matrix = (generate_traffic_matrix(non_uni).round(decimals=4)*10000).astype(dtype=int)

In [ ]:
non_uniform_matrix.T = np.rot90(np.fliplr(non_uniform_matrix))

In [ ]:
non_uniform_matrix

In [ ]:
udf = pd.DataFrame(data=uniform_matrix[:,:],    # values
        index=node_ids,    
        columns=node_ids)  
nudf = pd.DataFrame(data=non_uniform_matrix[:,:],    # values
        index=node_ids,    
        columns=node_ids)  

In [ ]:
udf.to_latex(buf="results/uniform_traffic_matrix.txt")

In [ ]:
nudf.to_latex(buf="results/non_uniform_traffic_matrix.txt")

In [ ]:
np.sum(generate_traffic_matrix(grav))

In [ ]:
np.sum(generate_traffic_matrix(uni))

In [ ]:
np.sum(generate_traffic_matrix(non_uni))

In [ ]:
sum(grav)

In [ ]:
sum(uni)

In [ ]:
sum(non_uni)

In [ ]:
traffic_dis = np.array([[0, 2, 1, 1, 1, 4, 1, 1, 2, 1, 1, 1, 1, 1],
                      [2, 0, 2, 1, 8, 2, 1, 5, 3, 5, 1, 5, 1, 4],
                      [1, 2, 0, 2, 3, 2, 11, 20, 5, 2, 1, 1, 1, 2],
                      [1, 1, 2, 0, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2],
                      [1, 8, 3, 1, 0, 3, 3, 7, 3, 3, 1, 5, 2, 5],
                      [4, 2, 2, 1, 3, 0, 2, 1, 2, 2, 1, 1, 1, 2],
                      [1, 1, 11, 2, 3, 2, 0, 9, 4, 20, 1, 8, 1, 4],
                      [1, 5, 20, 1, 7, 1, 9, 0, 27, 7, 2, 3, 2, 4],
                      [2, 3, 5, 2, 3, 2, 4, 27, 0, 75, 2, 9, 3, 1],
                      [1, 5, 2, 2, 3, 2, 20, 7, 75, 0, 1, 1, 2, 1],
                      [1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 2, 1, 61],
                      [1, 5, 1, 2, 5, 1, 8, 3, 9, 1, 2, 0, 1, 81],
                      [1, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 1, 0, 2],
                      [1, 4, 2, 2, 5, 2, 4, 4, 0, 1, 61, 81, 2, 0]])

In [ ]:
np.sum(traffic_dis*1e-3)

In [ ]:
cmdf = pd.DataFrame(data=traffic_dis,    # values
        index=node_ids,    
        columns=node_ids)  

In [ ]:
cmdf.to_latex(buf="results/chen_traffic_matrix.txt")

**Population-based traffic matrix NSFNET**

In [ ]:
traffic_matrix = loadmat("results/T_NASFnet14.mat")["T"]

In [ ]:
np.sum(traffic_matrix)

In [ ]:
traffic_matrix

In [ ]:
node_probs_pop = traffic_mat_to_node_probs(traffic_matrix, 14)

In [ ]:
np.sum(node_probs_pop)

In [ ]:
# pickle.dump(node_probs_pop, open("/Users/joshnevin/RL_FOCSLab/node_req_probs/nsfnet_population.pkl",'wb'))

**Population-based traffic matrix Google B4**

In [ ]:
traffic_matrix_gb4 = loadmat("results/GoogleB4_updated.mat")["T"]

In [ ]:
np.sum(traffic_matrix_gb4)

In [ ]:
node_probs_pop_gb4 = traffic_mat_to_node_probs(traffic_matrix_gb4, 12)

In [ ]:
len(node_probs_pop_gb4)

In [ ]:
np.sum(node_probs_pop_gb4)

In [ ]:
pickle.dump(node_probs_pop_gb4, open("/Users/joshnevin/RL_FOCSLab/node_req_probs/nsfnet_population_gb4.pkl",'wb'))

Round and generate LaTex table:

In [ ]:
traffic_matrix_rounded = (traffic_matrix.round(decimals=4)*1e4).astype(dtype=int)

In [ ]:
node_ids = np.arange(1,15,1)
pmdf = pd.DataFrame(data=traffic_matrix_rounded,    # values
        index=node_ids,    
        columns=node_ids)  
pmdf.to_latex(buf="results/population_traffic_matrix.txt")

Matplotlib color stuff:

In [ ]:
class MplColorHelper:

  def __init__(self, cmap_name, start_val, stop_val):
    self.cmap_name = cmap_name
    self.cmap = plt.get_cmap(cmap_name)
    self.norm = matplotlib.colors.Normalize(vmin=start_val, vmax=stop_val)
    self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

  def get_rgb(self, val):
    return self.scalarMap.to_rgba(val)

In [ ]:
# plt.rcParams["figure.figsize"] = [7.00, 3.50]
# plt.rcParams["figure.autolayout"] = True
x = np.arange(12)
y = x**2 + 5
# c = np.random.rand(len(x)) * 3 + 1.5
# df = pd.DataFrame({"x": x, "y": y, "c": c})
df = pd.DataFrame({"x": x, "y": y})

fig, ax = plt.subplots()
cmap = plt.cm.hot
norm = colors.Normalize(vmin=np.min(y), vmax=np.max(y))
ax.scatter(df.x, df.y, color=cmap(norm(df.y.values)))
ax.set_xticks(df.x)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
fig.colorbar(sm)
# plt.ylim([min(y),max(y)])
plt.show()

In [ ]:
COL = MplColorHelper('hot', np.min(y), np.max(y))

In [ ]:
COL.get_rgb(y)

In [ ]:
colours_dict = {"local_optima": result_m_shgo.xl}
# savemat("opt_results/"+env_id+str(int(threshold))+kernel+".mat", local_opt_dict)
savemat("results/rgb_link_utils.mat", local_opt_dict)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df.x, df.y, color=COL.get_rgb(y))
ax.set_xticks(df.x)
plt.show()

In [ ]:
y

In [ ]:
np.min(y)

In [ ]:
np.max(y)

In [ ]:
plt.plot(x,y)